Install depdenencies

In [ ]:
#!DEBIAN_FRONTEND=noninteractive
!sudo apt-get update # && sudo apt-get -y upgrade
!sudo apt-get -y install libegl1 
!sudo apt-get -y install libopengl0
!sudo apt-get -y install libxcb-cursor0
!sudo -v && wget -nv -O- https://download.calibre-ebook.com/linux-installer.sh | sudo sh /dev/stdin
!sudo apt-get install -y ffmpeg
!pip install tts pydub nltk beautifulsoup4 ebooklib tqdm
!pip install numpy==1.26.4

Download modified ebook2audiobookXTTS
https://github.com/Rihcus/ebook2audiobookXTTS

Origional unmodified version
https://github.com/DrewThomasson/ebook2audiobookXTTS

In [ ]:
!git clone https://github.com/Rihcus/ebook2audiobookXTTS

(optional) Uploading your own epub book.

By default this notebook will use a sample epub book for testing/demo. 

If you want to use your own book you will need to create a private kaggle data set, upload your epub to it, attach it to this notebook, and uncomment the two lines of code bellow, and update the data set path

In [ ]:
# !cp -r /kaggle/input/<name of your attached dataset>/*.epub /kaggle/working/ebook2audiobookXTTS #copy your custom book
# !rm /kaggle/working/ebook2audiobookXTTS/demo_mini_story_chapters_Drew.epub #remove default sample book

This to install xtts_v2 models

In [ ]:
import os
os.environ["COQUI_TOS_AGREED"] = "1"

!cd /kaggle/working/ebook2audiobookXTTS && tts --model_name tts_models/multilingual/multi-dataset/xtts_v2 --text "test" --speaker_wav ./4.wav --language_idx en --use_cuda true

This is a modified version of ebook2audiobookXTTS. 

- p1.py only runs the first part ebook2audiobookXTTS and generates chapter txts (I commented out other parts)
    - https://github.com/Rihcus/ebook2audiobookXTTS/blob/main/p1.py
- Worker_2T4.sh as a basic attempt at multigpu support. The 4 argument processes of ebook2audiobook will be run in parallel
    - Worker_2T4 will try to divide the chapter in even groups based on number of workers (ex 4 group 4 workers)
    - It will try to divy up the work between kaggles two T4 GPUS
    - I'm not sure how much of a difference it makes since kaggles cpu limitations
    
https://github.com/Rihcus/ebook2audiobookXTTS/blob/main/Worker_2T4.sh

https://github.com/Rihcus/ebook2audiobookXTTS/blob/main/p2a_worker_gpu1.py

https://github.com/Rihcus/ebook2audiobookXTTS/blob/main/p2a_worker_gpu2.py

In [ ]:
!cd /kaggle/working/ebook2audiobookXTTS && python p1.py "$(ls ./*.epub)" "4.wav" "en"
!cd /kaggle/working/ebook2audiobookXTTS && bash Worker_2T4.sh 4

p3.py runs the final ffmpeg command. ffmpeg has been a bit buggy
https://github.com/Rihcus/ebook2audiobookXTTS/blob/main/p3.py

In [ ]:
!cd /kaggle/working/ebook2audiobookXTTS && python p3.py "$(ls ./*.epub)" "4.wav" "en"